In [6]:
# imports
import numpy as np
import pandas as pd

# from getpass import getpass
# from dotenv import load_dotenv

In [14]:
# imports
import requests as re
import os

In [13]:
import json
from pandas.io.json import json_normalize  

In [38]:
import sqlite3
from sqlite3 import Error

In [15]:
yelpEP = 'https://api.yelp.com/v3/businesses/search'
# origin = '49.273085,-123.100375' # Coordinates for Main Street Station
origin = '49.283335,-123.116417' # Coordinates for Granville Station


# Examples from Postman:
# Yelp: https://api.yelp.com/v3/businesses/search?latitude=49.273085&longitude=-123.100375&term=coworking
# Custom function
def get_venues(coordinates,urlEP='https://api.foursquare.com/v3/places/search',category='11128',radius=10000):
    """Parameters:
    - Coordinates: (string) Latitude and longitude, separated by a comma.
    - Term: (string, optional) Search term to use. Default is 'coworking'.
    - urlEP: (string, optional) Root for API endpoint URL. Default for Foursquare Places.
    - Category: (string, optional) Category of place. If blank default is 11128 
    for 'Business and Professional Services > Office > Coworking Space'
        - 13003 for 'Dining and Drinking > Bar'. 
    - Radius: (integer, optional) Radius of search results.
    
    Returns:
    - URL for API call (string)
    """
    urlEP=str(urlEP)
    radius = int(radius)
    category=str(category)

    if urlEP.find('api.yelp') > 0:
        coordinates = str.split(coordinates,sep=',')
        urlAppend = f'?latitude={coordinates[0]}&longitude={coordinates[1]}&radius={radius}&term={category}' # yelp
    else:
        urlAppend = f'?ll={coordinates}&radius={radius}&categories={category}' # Default is for foursquare
    request_URL=urlEP+urlAppend
    
    return request_URL

requestURL_4squarePlaces = get_venues(origin)
requestURL_yelp = get_venues(origin,yelpEP,category='coworking')
print(requestURL_4squarePlaces)
print(requestURL_yelp)

https://api.foursquare.com/v3/places/search?ll=49.283335,-123.116417&radius=10000&categories=11128
https://api.yelp.com/v3/businesses/search?latitude=49.283335&longitude=-123.116417&radius=10000&term=coworking


In [16]:
#set the keys
key4square = os.environ["API_FOURSQUARE"]
keyYelp = os.environ['API_Yelp']

# Test that the keys work
# print(key4square)
# print(keyYelp)

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [ ]:

# Four square GET request
header_4square = {"Accept": "application/JSON"}
header_4square['Authorization'] = key4square
print(header_4square)
response4squarePlaces = re.get(requestURL_4squarePlaces, headers=header_4square) ## API request
# print(response4squarePlaces.json())

In [54]:
# Turn GET request to JSON data: Four Square
json4squarePlaces = response4squarePlaces.json()

Parse through the response to get the POI details you want (rating, name, location, etc)

In [55]:
foursquare = pd.DataFrame(json4squarePlaces['results'])
foursquare.head(3)

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone
0,4cbdb4d59552b60c758aec8b,"[{'id': 11128, 'name': 'Coworking Space', 'ico...",[],93,"{'main': {'latitude': 49.282736, 'longitude': ...",/v3/places/4cbdb4d59552b60c758aec8b,"{'address': '555 Seymour St', 'country': 'CA',...",BC Campus,{},America/Vancouver
1,4d0fba4bc3dc37045a663474,"[{'id': 11128, 'name': 'Coworking Space', 'ico...",[],270,"{'main': {'latitude': 49.281157, 'longitude': ...",/v3/places/4d0fba4bc3dc37045a663474,"{'address': '401 Georgia St W', 'country': 'CA...",401 West Georgia,{'children': [{'fsq_id': '62c712d232c44b48aef6...,America/Vancouver
2,4ffc942de4b0990d47bb425d,"[{'id': 11128, 'name': 'Coworking Space', 'ico...",[],280,"{'main': {'latitude': 49.284754, 'longitude': ...",/v3/places/4ffc942de4b0990d47bb425d,"{'address': '602 Hastings St W', 'address_exte...",Collaborative Open Space Vancouver,{},America/Vancouver


In [86]:
foursquare.columns

Index(['fsq_id', 'categories', 'chains', 'distance', 'geocodes', 'link',
       'location', 'name', 'related_places', 'timezone'],
      dtype='object')

Put your parsed results into a DataFrame

In [56]:
# use a function from Pandas json_normalize()
# dfFoursquare = json_normalize(json4squarePlaces['results'], record_path='categories')
# dfFoursquare = json_normalize(json4squarePlaces['results'], meta=['fsq_id','distance','geocodes', 'link','location', 'name'])
dfFoursquare = json_normalize(json4squarePlaces, record_path='results')
# print(dfFoursquare.columns)
dfFoursquare.head(3)

C:\Users\silvh\AppData\Local\Temp\ipykernel_3940\477756026.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  dfFoursquare = json_normalize(json4squarePlaces, record_path='results')


,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.address,...,location.locality,location.region,geocodes.roof.latitude,geocodes.roof.longitude,location.postcode,related_places.children,location.address_extended,related_places.parent.fsq_id,related_places.parent.name,location.neighborhood
0,4cbdb4d59552b60c758aec8b,"[{'id': 11128, 'name': 'Coworking Space', 'ico...",[],93,/v3/places/4cbdb4d59552b60c758aec8b,BC Campus,America/Vancouver,49.282736,-123.115513,555 Seymour St,...,Vancouver,BC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4d0fba4bc3dc37045a663474,"[{'id': 11128, 'name': 'Coworking Space', 'ico...",[],270,/v3/places/4d0fba4bc3dc37045a663474,401 West Georgia,America/Vancouver,49.281157,-123.115053,401 Georgia St W,...,Vancouver,BC,49.281157,-123.115053,V6B 5A1,"[{'fsq_id': '62c712d232c44b48aef66fe4', 'name'...",NaN,NaN,NaN,NaN
2,4ffc942de4b0990d47bb425d,"[{'id': 11128, 'name': 'Coworking Space', 'ico...",[],280,/v3/places/4ffc942de4b0990d47bb425d,Collaborative Open Space Vancouver,America/Vancouver,49.284754,-123.113251,602 Hastings St W,...,Vancouver,BC,49.284754,-123.113251,V6B 1P2,NaN,# 728,NaN,NaN,NaN


In [116]:
# this is the list of Foursquare categories in the results
json_normalize(json4squarePlaces['results'], record_path='categories')

C:\Users\silvh\AppData\Local\Temp\ipykernel_16800\389920604.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  json_normalize(json4squarePlaces['results'], record_path='categories')


,id,name,icon.prefix,icon.suffix
0,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
1,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
2,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
3,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
4,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
5,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
6,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
7,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
8,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png
9,11128,Coworking Space,https://ss3.4sqi.net/img/categories_v2/buildin...,.png


In [ ]:
# This time, create GET request with ratings parameter included
# Examples from Postman:
# Yelp: https://api.yelp.com/v3/businesses/search?latitude=49.273085&longitude=-123.100375&term=coworking
# Custom function
def get_venuesRated(coordinates,urlEP='https://api.foursquare.com/v3/places/search',category='11128',radius=10000):
    """Parameters:
    - Coordinates: (string) Latitude and longitude, separated by a comma.
    - Term: (string, optional) Search term to use. Default is 'coworking'.
    - urlEP: (string, optional) Root for API endpoint URL. Default for Foursquare Places.
    - Category: (string, optional) Category of place. If blank default is 11128 
    for 'Business and Professional Services > Office > Coworking Space'
        - 13003 for 'Dining and Drinking > Bar'. 
    - Radius: (integer, optional) Radius of search results.
    
    Returns:
    - URL for API call (string)
    """
    urlEP=str(urlEP)
    radius = int(radius)
    category=str(category)

    if urlEP.find('api.yelp') > 0:
        coordinates = str.split(coordinates,sep=',')
        urlAppend = f'?latitude={coordinates[0]}&longitude={coordinates[1]}&radius={radius}&term={category}' # yelp
    else:
        urlAppend = f'?ll={coordinates}&radius={radius}&categories={category}&fields=rating' # Default is for foursquare
    request_URL=urlEP+urlAppend
    
    return request_URL

requestURL_4squarePlaces2 = get_venuesRated(origin)
print(requestURL_4squarePlaces2)

https://api.foursquare.com/v3/places/search?ll=49.283335,-123.116417&radius=10000&categories=11128&fields=rating
https://api.yelp.com/v3/businesses/request?latitude=49.283335&longitude=-123.116417&radius=10000&term=coworking


In [119]:
# This time with ratings included
# Four square GET request
header_4square = {"Accept": "application/JSON"}
header_4square['Authorization'] = key4square
print(header_4square)
response4squarePlaces2 = re.get(requestURL_4squarePlaces, headers=header_4square) ## API request
print(response4squarePlaces2.json())

{'Accept': 'application/JSON', 'Authorization': 'fsq3f2XsuYKnioaDoty53EOHDgCyE22ecgw6D/HW/02QWCs='}
{'results': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}], 'context': {'geo_bounds': {'circle': {'center': {'latitude': 49.283335, 'longitude': -123.116417}, 'radius': 10000}}}}


In [ ]:
## The GET request resulted in empty results 

# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [17]:
# Yelp GET request
headerYelp = {"Accept": "application/JSON"}
headerYelp['Authorization'] = 'BEARER '+keyYelp
# print(headerYelp)
responseYelp = re.get(requestURL_yelp, headers=headerYelp) ## API request
responseYelp.status_code


200

In [19]:
# Turn GET request to JSON data: Yelp
jsonYelp = responseYelp.json()
jsonYelp

{'businesses': [{'id': 'JeUBI8BdKPlzgkeq8Pn1vg',
   'alias': 'l-atelier-vancouver-vancouver-4',
   'name': "L'Atelier Vancouver",
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/dB3HkyLXCe75-jqlYgeE6g/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/l-atelier-vancouver-vancouver-4?adjust_creative=VD-0MX-Cz4siaXaVtHPotQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=VD-0MX-Cz4siaXaVtHPotQ',
   'review_count': 3,
   'categories': [{'alias': 'sharedofficespaces',
     'title': 'Shared Office Spaces'}],
   'rating': 5.0,
   'coordinates': {'latitude': 49.28322, 'longitude': -123.11008},
   'transactions': [],
   'location': {'address1': '319 W Hastings Street',
    'address2': '',
    'address3': '',
    'city': 'Vancouver',
    'zip_code': 'V6B 1H6',
    'country': 'CA',
    'state': 'BC',
    'display_address': ['319 W Hastings Street',
     'Vancouver, BC V6B 1H6',
     'Canada']},
   'phone': '+12368805489',
   'display_phone': '+1 236-

Parse through your result and get POI details

In [26]:
print(yelp.shape)
print(yelp.columns)

(20, 15)
Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'coordinates', 'transactions', 'location',
       'phone', 'display_phone', 'distance'],
      dtype='object')


Put your parsed results into a DataFrame

In [35]:
dfYelp = json_normalize(jsonYelp,record_path='businesses')
dfYelp.head(3)

C:\Users\silvh\AppData\Local\Temp\ipykernel_3940\4048724908.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  dfYelp = json_normalize(jsonYelp,record_path='businesses')


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,JeUBI8BdKPlzgkeq8Pn1vg,l-atelier-vancouver-vancouver-4,L'Atelier Vancouver,https://s3-media1.fl.yelpcdn.com/bphoto/dB3Hky...,False,https://www.yelp.com/biz/l-atelier-vancouver-v...,3,"[{'alias': 'sharedofficespaces', 'title': 'Sha...",5.0,[],...,49.283220,-123.110080,319 W Hastings Street,,,Vancouver,V6B 1H6,CA,BC,"[319 W Hastings Street, Vancouver, BC V6B 1H6,..."
1,7i-PXkf-vxBGcWGKagQOYg,the-profile-vancouver-2,The Profile,https://s3-media3.fl.yelpcdn.com/bphoto/8O17t4...,False,https://www.yelp.com/biz/the-profile-vancouver...,4,"[{'alias': 'sharedofficespaces', 'title': 'Sha...",3.0,[],...,49.283837,-123.105446,55 Water Street,Suite 408,,Vancouver,V6B 1A1,CA,BC,"[55 Water Street, Suite 408, Vancouver, BC V6B..."
2,7iVmZWviiyRZC-5sIXdrQw,telus-garden-vancouver,TELUS Garden,https://s3-media1.fl.yelpcdn.com/bphoto/vLTrac...,False,https://www.yelp.com/biz/telus-garden-vancouve...,2,"[{'alias': 'apartments', 'title': 'Apartments'...",4.0,[],...,49.280450,-123.118040,777 Richards Street,,,Vancouver,V6B 0M3,CA,BC,"[777 Richards Street, Vancouver, BC V6B 0M3, C..."


In [36]:
print(dfYelp.shape)
print(dfYelp.columns)

(20, 23)
Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'phone', 'display_phone',
       'distance', 'coordinates.latitude', 'coordinates.longitude',
       'location.address1', 'location.address2', 'location.address3',
       'location.city', 'location.zip_code', 'location.country',
       'location.state', 'location.display_address'],
      dtype='object')


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [63]:
# Desired columns from Yelp:
columnsYelp = (
       'id','name','review_count', 'rating', 'distance', 'coordinates.latitude', 
       'coordinates.longitude', 'location.address1')
dfYelp1 = dfYelp.loc[:,columnsYelp]
dfYelp1.head(2)

,id,name,review_count,rating,distance,coordinates.latitude,coordinates.longitude,location.address1
0,JeUBI8BdKPlzgkeq8Pn1vg,L'Atelier Vancouver,3,5.0,462.234321,49.283220,-123.110080,319 W Hastings Street
1,7i-PXkf-vxBGcWGKagQOYg,The Profile,4,3.0,797.689461,49.283837,-123.105446,55 Water Street


In [74]:
# Desired columns from Four Square:
columns4square = (
       'fsq_id', 'distance', 'link', 'name', 'geocodes.main.latitude', 'geocodes.main.longitude',
       'location.address'

dfFoursquare1 = dfFoursquare.loc[:,columns4square]
dfFoursquare1.head(2)

,fsq_id,distance,link,name,geocodes.main.latitude,geocodes.main.longitude,location.address
0,4cbdb4d59552b60c758aec8b,93,/v3/places/4cbdb4d59552b60c758aec8b,BC Campus,49.282736,-123.115513,555 Seymour St
1,4d0fba4bc3dc37045a663474,270,/v3/places/4d0fba4bc3dc37045a663474,401 West Georgia,49.281157,-123.115053,401 Georgia St W


In [99]:
# Generate SQL script to make table: Four Square

tableName_foursquare = 'coworking4square'
sql_column_code = f'CREATE TABLE IF NOT EXISTS {tableName_foursquare} ('

for column in columns4square:
    column = column.replace('.','_')
    sql_column_code += create_sql_table_columns(column)
    print(create_sql_table_columns(column))
    if column == columns4square[-1].replace('.','_'):
        sql_column_code += ')'
    else:
        sql_column_code += ", "

sql_column_code




fsq_id text
distance text
link text
name text
geocodes_main_latitude text
geocodes_main_longitude text
location_address text


'CREATE TABLE IF NOT EXISTS coworking4square (fsq_id text, distance text, link text, name text, geocodes_main_latitude text, geocodes_main_longitude text, location_address text)'

In [40]:
# The following script creates a connection to the SQLite database:
def create_connection(path):
    connection = None
    try: # Uses .connect() from the sqlite3 module and takes the SQLite database path as a parameter
        connection = sqlite3.connect(path) # 
        print("Connection to SQLite DB successful")
    except Error as e: # catches any exception that might be thrown if .connect() fails to establish a connection.
        print(f"The error '{e}' occurred")

    return connection
# This function not necessary

In [41]:
# Create connection object 

connection = create_connection("sm_app.sqlite") # filename can be anything, just keep extension

Connection to SQLite DB successful


In [97]:
c = connection.cursor()
c.execute(sql_column_code)

dfFoursquare1.to_sql(tableName_foursquare, connection, if_exists='replace', index = False)
tableName_foursquare

'coworking4square'

In [125]:
# Make sure that SQL table can be retrieved
pd.read_sql_query(f'SELECT * FROM {tableName_foursquare}', connection)

,fsq_id,distance,link,name,geocodes.main.latitude,geocodes.main.longitude,location.address
0,4cbdb4d59552b60c758aec8b,93,/v3/places/4cbdb4d59552b60c758aec8b,BC Campus,49.282736,-123.115513,555 Seymour St
1,4d0fba4bc3dc37045a663474,270,/v3/places/4d0fba4bc3dc37045a663474,401 West Georgia,49.281157,-123.115053,401 Georgia St W
2,4ffc942de4b0990d47bb425d,280,/v3/places/4ffc942de4b0990d47bb425d,Collaborative Open Space Vancouver,49.284754,-123.113251,602 Hastings St W
3,4c2da9441683b713e66726ec,337,/v3/places/4c2da9441683b713e66726ec,The Network Hub,49.283982,-123.111870,422 Richards St
4,512bbcb0e4b0173c05eb92c8,352,/v3/places/512bbcb0e4b0173c05eb92c8,Wave Maker Marketing,49.286489,-123.116887,400 Burrard St
5,4e527c4314955fcafdc86820,367,/v3/places/4e527c4314955fcafdc86820,Commerce Place - the Eminata Group,49.286536,-123.117297,400 Burrard St
6,5b35d0e031ac6c0039a23c12,380,/v3/places/5b35d0e031ac6c0039a23c12,WeWork,49.285363,-123.112192,333 Seymour St
7,591375aa6a8d866b5410bb7e,400,/v3/places/591375aa6a8d866b5410bb7e,WeWork,49.286253,-123.119950,595 Burrard St
8,4b14084cf964a5201e9c23e3,400,/v3/places/4b14084cf964a5201e9c23e3,Small Biz Education Centre,49.285495,-123.111772,601 Cordova St W
9,4c43905f36d6a5939c5869a8,534,/v3/places/4c43905f36d6a5939c5869a8,Leckie Building,49.283801,-123.109194,220 Cambie St


----


In [ ]:
# Desired columns from Yelp:
columnsYelp = (
       'id','name','review_count', 'rating', 'distance', 'coordinates.latitude', 
       'coordinates.longitude', 'location.address1')
dfYelp1 = dfYelp.loc[:,columnsYelp]
dfYelp1.head(2)

,id,name,review_count,rating,distance,coordinates.latitude,coordinates.longitude,location.address1
0,JeUBI8BdKPlzgkeq8Pn1vg,L'Atelier Vancouver,3,5.0,462.234321,49.283220,-123.110080,319 W Hastings Street
1,7i-PXkf-vxBGcWGKagQOYg,The Profile,4,3.0,797.689461,49.283837,-123.105446,55 Water Street


In [101]:
# Generate SQL script to make table: Four Square
# Create a function

def sql_table_code(tableName,tableColumns):
    """Generate the SQL code to create a table: 
    Params:
    - tableName (string): Enter the name of the table to be created in the database.
    - tableColumns: A list/tuple of the columns to be created
    """
    sql_column_code = f'CREATE TABLE IF NOT EXISTS {tableName} ('
    for column in tableColumns:
        column = column.replace('.','_')
        sql_column_code += f'{column} text'
        if column == tableColumns[-1].replace('.','_'):
            sql_column_code += ')'
        else:
            sql_column_code += ", "
    return sql_column_code

sql_table_code4Square = sql_table_code('coworkingYelp',columnsYelp)
sql_table_code4Square

'CREATE TABLE IF NOT EXISTS coworkingYelp (id text, name text, review_count text, rating text, distance text, coordinates_latitude text, coordinates_longitude text, location_address1 text)'

In [134]:
# Create SQL table for Yelp coworking results
dfYelp1.to_sql('coworkingYelp', connection, if_exists='replace', index = False)

20

In [135]:
# Make sure that SQL table can be retrieved
pd.read_sql_query(f'SELECT * FROM coworkingYelp', connection)

,id,name,review_count,rating,distance,coordinates.latitude,coordinates.longitude,location.address1
0,JeUBI8BdKPlzgkeq8Pn1vg,L'Atelier Vancouver,3,5.0,462.234321,49.283220,-123.110080,319 W Hastings Street
1,7i-PXkf-vxBGcWGKagQOYg,The Profile,4,3.0,797.689461,49.283837,-123.105446,55 Water Street
2,7iVmZWviiyRZC-5sIXdrQw,TELUS Garden,2,4.0,313.470364,49.280450,-123.118040,777 Richards Street
3,mpESKGpsgGMzeXP0wmOB0w,The Network Hub,35,4.5,318.503564,49.283692,-123.112060,422 Richards Street
4,oGigV7Ad6UTUjkYQcuCZ0g,HiVE Vancouver,6,4.0,640.393096,49.282105,-123.107792,210-128 W Hastings Street
5,JnqaBvmmVoSkz64vCB89-w,Offices In the Marine Building,1,5.0,450.503075,49.287360,-123.117129,355 Burrard Street
6,BJJVkh3g4pgX9ftrFAi0KA,Central Park Business Centre,1,5.0,8747.660928,49.232670,-123.024200,3665 Kingsway
7,1V5ztK_AyUZjlolRfTX_WA,Jameson Offices,1,5.0,323.772812,49.286186,-123.115517,838 W Hastings Street
8,DwO0tlgrmibhgOQCklR5nA,Penthouse Event & Business Suite,7,4.0,1916.717858,49.272442,-123.095711,333 Terminal Avenue
9,Nn1RVmYpuO2M8Yw_JNaefA,WeWork,8,5.0,402.950903,49.286219,-123.119780,595 Burrard Street


In [ ]:
sql_table_code4Square

Get the top 10 restaurants according to their rating

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).